# AI Image Classifier
- **TensorFlow:** to use a pretrained machine learning model that can look at an image and classify it
- **OpenCV:** to allow manipulations on the image so it can be passed to TensorFlow
- **MobileNetV2:** is a pre-trained model

**[YouTube](https://www.youtube.com/watch?v=XZdY15sHUa8)**

- `uv run streamlit run main.py`

### Step 1: Load the pre-trained MobileNetV2 model
### Step 2: Prepare and Preprocess the image for MobileNetV2
- Convert the image inot an Array of numbers
- Resize the image
- Preprocess the image
- Expand dimensions to create a batch of images

In [ ]:
import cv2
import numpy as np # comes with tensorflow by default
import streamlit as st
from tensorflow.keras.applications.mobilenet_v2 import (
    MobileNetV2,
    preprocess_input,
    decode_predictions
)
from PIL import Image # come with cv2 by default
#===============================================================================================================================================
# tensorflow is a deep learning framework
# keras is a high-level API for building and training deep learning models, included in tensorflow
# tensorflow and keras are used for building and training the image classification model
# this project will use a modelthat already has been pre-trained on a large dataset (ImageNet) and fine-tune it for our specific use case
# model: MobileNetV2, a lightweight model suitable for mobile and embedded vision applications
# it will be downloaded automatically when we load the model
#===============================================================================================================================================

def load_model():
    # TODO 1: Load the pre-trained MobileNetV2 model
    #===============================================================================================================================================
    # MobileNetV2 is a pre-trained and we bring in the model architecture and weights to use it for our image classification task
    #===============================================================================================================================================
    # MobileNetV2 is a convolutional neural network architecture that is optimized for mobile and embedded vision applications
    # it uses specific techniques and artchitecture to look at images efficiently while maintaining good accuracy
    # weights="imagenet": load the model with pre-trained weights on the ImageNet dataset
    # weights: the learned parameters(values) of the model that determine how it processes input data to make predictions
    # weights are uniques numbers that the model learns during training, equal a particular output for a given input
    #===============================================================================================================================================
    model = MobileNetV2(weights="imagenet")
    return model

#======================================================================
# this model takes an image but not just any image
# it has to be the correct size and format for the model to understand it
# this function takes in a image and preprocesses it for the model
# so the model can understand and make predictions on it
#=======================================================================
def preprocess_image(image):

    # TODO 2: Preprocess the image for MobileNetV2
    #======================================================================
    # TODO 2-1: Convert the image inot an Array of numbers
    # convert the image to a numpy array of numbers
    # each number represents a pixel value in the image
    # each pixel represented by its red, green, blue values [R,G,B]
    #[[1,3,5],[3,8,1],[6,1,2]] this is a 3x3 image with RGB values
    #======================================================================
    img = np.array(image)

    # TODO 2-2: Resize the image
    # image needs to be resized and preprocessed
    img = cv2.resize(img, (224, 224))# resize the image to 224x224 pixels (the size the model expects)

    # TODO 2-3: Preprocess the image
    # images need to be preprocessed before being fed into the model
    # preprocessing involves scaling pixel values to a specific range
    # and normalizing the image data to match the distribution of the training data
    #======================================================================
    img = preprocess_input(img)# preprocess the image using the preprocess_input function from MobileNetV2

    # TODO 2-4: Expand dimensions to create a batch of images
    # image needs to be in a batch format
    #======================================================================
    # takes a single image and converts it into a batch of images
    # model expects a batch of images as input, even if it's just one image
    #======================================================================
    img = np.expand_dims(img, axis=0)# add an extra dimension to the image array to create a batch of size 1
    
    return img

### Step 3: Classify Image and Decode Predictions
- pass the preprocess image to the model and get predictions

In [ ]:
import cv2
import numpy as np # comes with tensorflow by default
import streamlit as st
from tensorflow.keras.applications.mobilenet_v2 import (
    MobileNetV2,
    preprocess_input,
    decode_predictions
)
from PIL import Image # come with cv2 by default

def load_model():
    model = MobileNetV2(weights="imagenet")
    return model

def preprocess_image(image):
    img = np.array(image)
    img = cv2.resize(img, (224, 224))
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    return img

#===========================================================================================================
# TODO 3: Classify Image and Decode Predictions
# pass the model a image
# then it gives us back predictions, which is and array of numeric values, [0.34, 0.1, 0.56, ...]
# each value represent a percentage confidence score for each class that the model can predict
# the index of each value corresponds to a specific class label, such as "cat", "dog", "car", etc.
# this means that [0.34, 0.1, 0.56, ...] could mean that 
# the model is 34% confident that the image is a "cat", 10% confident it's a "dog", and 56% confident it's a "car" etc...
# the index of the highest value in the array corresponds to the class that the model thinks the image belongs to
# decode_predictions() function takes these numeric predictions and converts them into human-readable labels
#===========================================================================================================
def classify_image(model, image):
    try:
        processed_image = preprocess_image(image)
        predictions = model.predict(processed_image)

        # takes the numeric prdiction that are genrated by the model and converts them into human-readable labels
        decoded_predictions = decode_predictions(predictions, top=3)[0]# takes the top 3 predictions, ones with the highest confidence scores
        
        return decoded_predictions
    except Exception as e:
        st.error(f"Error classifying image: {str(e)}")
        return None

### Step 4: Create The Streamlit App

In [ ]:

# Create The Streamlit App
def main():
    st.set_page_config(page_title="AI Image Classifier", page_icon="🖼️", layout="centered")
    
    st.title("AI Image Classifier")
    st.write("Upload an image and let AI tell you what is in it!")
    

    #==============================================================
    # In streamlit, caching is used to store the results of expensive computations
    # so that they don't have to be recomputed every time the app reruns
    # so resources that are used often can be cached to improve performance
    # and they don't have to be reloaded every time
    #==============================================================                         
    # this will cache the model so it doesn't reload every time
    @st.cache_resource
    def load_cached_model():
        return load_model()
    
    model = load_cached_model()
    
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png"])
    
    if uploaded_file is not None:
        # display the uploaded image on the app
        # use_container_width=True makes the image responsive to the container width
        image = st.image(
            uploaded_file, caption="Uploaded Image", use_container_width=True
        )
        btn = st.button("Classify Image")
        
        if btn:
            with st.spinner("Analyzing Image..."):# loading spinner while analyzing
                image = Image.open(uploaded_file)# open the uploaded image file
                predictions = classify_image(model, image)# classify the image
                
                if predictions:
                    st.subheader("Predictions")
                    for _, label, score in predictions: # (index, label, score)
                        st.write(f"**{label}**: {score:.2%}")
                        